In [1]:
#Import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import datetime

In [2]:
#Load data matrix
tickerdata = pd.read_excel(r'Data/Processed Data Matrix for Thesis.xlsx',engine='openpyxl',dtype=str)
tickerdata['CI'] = pd.to_numeric(tickerdata['CI'])
tickerdata['ISF'] = pd.to_numeric(tickerdata['ISF'])
tickerdata.head()

,Sharing Economy Firm,Incumbent Firms,Ticker,Exchange,CI,Market Index,ISF,Start(t=1),End(t=1),Start(t=3),...,SP(Start(t=3)),SP(End(t=3)),SP(Start(t=6)),SP(End(t=6)),SP(Start(t=12)),SP(End(t=12)),SP(Start(t=24)),SP(End(t=24)),SP(Start(t=36)),SP(End(t=36))
0,Airbnb,Marriott International,MAR,NASDAQ,1,IXIC,0,2008-07-31 00:00:00,2008-08-31 00:00:00,2008-05-31 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Airbnb,Marriott International,MAR,NASDAQ,1,IXIC,1,2008-08-31 00:00:00,2008-09-30 00:00:00,2008-08-31 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Airbnb,iShares Europe ETF,IEV,NYSEARCA,0,NYA,0,2008-07-31 00:00:00,2008-08-31 00:00:00,2008-05-31 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Airbnb,iShares Europe ETF,IEV,NYSEARCA,0,NYA,1,2008-08-31 00:00:00,2008-09-30 00:00:00,2008-08-31 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Bird,Dorel Industries Inc Class B,DIIBF,OTCMKTS,1,OTCQX,0,2017-08-30 00:00:00,2017-09-30 00:00:00,2017-06-30 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
#Reshape/melt data matrix to make time an attribute
transformedtickerdata = tickerdata.melt(id_vars=['Sharing Economy Firm', 'Incumbent Firms', 'Ticker',
       'Exchange', 'CI', 'Market Index', 'ISF', 'SP(Start(t=1))', 'SP(End(t=1))', 'SP(Start(t=3))',
       'SP(End(t=3))', 'SP(Start(t=6))', 'SP(End(t=6))', 'SP(Start(t=12))', 'SP(End(t=12))', 'SP(Start(t=24))', 'SP(End(t=24))', 'SP(Start(t=36))', 'SP(End(t=36))'],var_name='t',value_name='Date').drop(columns=['SP(Start(t=1))', 'SP(End(t=1))', 'SP(Start(t=3))',
       'SP(End(t=3))', 'SP(Start(t=6))', 'SP(End(t=6))', 'SP(Start(t=12))', 'SP(End(t=12))', 'SP(Start(t=24))', 'SP(End(t=24))', 'SP(Start(t=36))', 'SP(End(t=36))'])
transformedtickerdata['Ticker'] = transformedtickerdata['Ticker'].astype(str)
transformedtickerdata.tail()

,Sharing Economy Firm,Incumbent Firms,Ticker,Exchange,CI,Market Index,ISF,t,Date
1531,Facebook,Gartner,IT,NYSE,0,NYA,1,End(t=36),2007-02-28 00:00:00
1532,Twitter,Viacom,VIAC,NASDAQ,1,IXIC,0,End(t=36),2006-03-31 00:00:00
1533,Twitter,Viacom,VIAC,NASDAQ,1,IXIC,1,End(t=36),2009-03-31 00:00:00
1534,Twitter,Banc of California Inc,BANC,NYSE,0,NYA,0,End(t=36),2006-03-31 00:00:00
1535,Twitter,Banc of California Inc,BANC,NYSE,0,NYA,1,End(t=36),2009-03-31 00:00:00


In [4]:
#Load equity data and transform
equitydata = pd.read_excel(r'Data/Bloomberg Equity Data.xlsx', engine='openpyxl',usecols=[0,1,3],sheet_name=None,dtype=str)
datalist = []
for ticker, data in equitydata.items():
    data['Ticker'] = str(ticker)
    datalist.append(data)
transformedequitydata = pd.concat(datalist,axis=0)
transformedequitydata.dropna(axis=1,how='all')
transformedequitydata['Last Price'] = pd.to_numeric(transformedequitydata['Last Price'])
transformedequitydata.head()

,Date,Last Price,Volume,Ticker
0,2011-09-29 00:00:00,27.0523,7118567,MAR
1,2011-09-28 00:00:00,26.2039,4077685,MAR
2,2011-09-27 00:00:00,26.8072,5198437,MAR
3,2011-09-26 00:00:00,26.1003,4718259,MAR
4,2011-09-25 00:00:00,25.6101,6045623,MAR


In [5]:
#Load index data and transform
indexdata = pd.read_excel(r'Data/Bloomberg Index Data.xlsx', engine='openpyxl',usecols=[0,1],sheet_name='S&P500',dtype=str)
indexdata['Last Price'] = pd.to_numeric(indexdata['Last Price'])
indexdata['Date'] = pd.to_datetime(indexdata['Date'])
indexdata.head()

,Date,Last Price
0,2021-03-12,3943.34
1,2021-03-11,3939.34
2,2021-03-10,3898.81
3,2021-03-09,3875.44
4,2021-03-08,3821.35


In [6]:
#Merge ticker data with empty matrix
merged_data = transformedtickerdata.merge(transformedequitydata, how='left',left_on=['Date','Ticker'],right_on=['Date','Ticker']).dropna(axis=1,how='all').rename(columns={'Last Price':'SP(t)','Volume':'Volume(t)'})
merged_data['Volume(t)'] = pd.to_numeric(merged_data['Volume(t)'])
merged_data['Date'] = pd.to_datetime(merged_data['Date'])
merged_data.reset_index()
merged_data.head()

,Sharing Economy Firm,Incumbent Firms,Ticker,Exchange,CI,Market Index,ISF,t,Date,SP(t),Volume(t)
0,Airbnb,Marriott International,MAR,NASDAQ,1,IXIC,0,Start(t=1),2008-07-31,24.1584,3808057
1,Airbnb,Marriott International,MAR,NASDAQ,1,IXIC,1,Start(t=1),2008-08-31,26.3029,2756931
2,Airbnb,iShares Europe ETF,IEV,NYSEARCA,0,NYA,0,Start(t=1),2008-07-31,47.3600,228441
3,Airbnb,iShares Europe ETF,IEV,NYSEARCA,0,NYA,1,Start(t=1),2008-08-31,45.4800,488027
4,Bird,Dorel Industries Inc Class B,DIIBF,OTCMKTS,1,OTCQX,0,Start(t=1),2017-08-30,23.0800,376


In [7]:
#Merge index data with rest of data
merged_data=merged_data.merge(indexdata,how='left',left_on=['Date'],right_on=['Date']).rename(columns={'Last Price':'S&P500(t)'})
merged_data.head()

,Sharing Economy Firm,Incumbent Firms,Ticker,Exchange,CI,Market Index,ISF,t,Date,SP(t),Volume(t),S&P500(t)
0,Airbnb,Marriott International,MAR,NASDAQ,1,IXIC,0,Start(t=1),2008-07-31,24.1584,3808057,1267.38
1,Airbnb,Marriott International,MAR,NASDAQ,1,IXIC,1,Start(t=1),2008-08-31,26.3029,2756931,1282.83
2,Airbnb,iShares Europe ETF,IEV,NYSEARCA,0,NYA,0,Start(t=1),2008-07-31,47.3600,228441,1267.38
3,Airbnb,iShares Europe ETF,IEV,NYSEARCA,0,NYA,1,Start(t=1),2008-08-31,45.4800,488027,1282.83
4,Bird,Dorel Industries Inc Class B,DIIBF,OTCMKTS,1,OTCQX,0,Start(t=1),2017-08-30,23.0800,376,2457.59


In [8]:
#Re-Pivot Data
complete_data = merged_data.pivot(index=['Sharing Economy Firm','Incumbent Firms','Ticker','Exchange','CI','Market Index','ISF'],columns=['t'])
complete_data.head()

Date  \
t                                                                                       End(t=1)   
Sharing Economy Firm Incumbent Firms              Ticker Exchange CI Market Index ISF              
Airbnb               Marriott International       MAR    NASDAQ   1  IXIC         0   2008-08-31   
                                                                                  1   2008-09-30   
                     iShares Europe ETF           IEV    NYSEARCA 0  NYA          0   2008-08-31   
                                                                                  1   2008-09-30   
Bird                 Dorel Industries Inc Class B DIIBF  OTCMKTS  1  OTCQX        0   2017-09-30   

                                                                                                  \
t                                                                                      End(t=12)   
Sharing Economy Firm Incumbent Firms              Ticker Exchange CI Market Index ISF              
Airbnb               Marriott International       MAR    NASDAQ   1  IXIC         0   2008-08-31   
                                                                                  1   2009-08-31   
                     iShares Europe ETF           IEV    NYSEARCA 0  NYA          0   2008-08-31   
                                                                                  1   2009-08-31   
Bird                 Dorel Industries Inc Class B DIIBF  OTCMKTS  1  OTCQX        0   2017-09-30   

                                                                                                  \
t                                                                                      End(t=24)   
Sharing Economy Firm Incumbent Firms              Ticker Exchange CI Market Index ISF              
Airbnb               Marriott International       MAR    NASDAQ   1  IXIC         0   2008-08-31   
                                                                                  1   2010-08-31   
                     iShares Europe ETF           IEV    NYSEARCA 0  NYA          0   2008-08-31   
                                                                                  1   2010-08-31   
Bird                 Dorel Industries Inc Class B DIIBF  OTCMKTS  1  OTCQX        0   2017-09-30   

                                                                                                  \
t                                                                                       End(t=3)   
Sharing Economy Firm Incumbent Firms              Ticker Exchange CI Market Index ISF              
Airbnb               Marriott International       MAR    NASDAQ   1  IXIC         0   2008-08-31   
                                                                                  1   2008-11-30   
                     iShares Europe ETF           IEV    NYSEARCA 0  NYA          0   2008-08-31   
                                                                                  1   2008-11-30   
Bird                 Dorel Industries Inc Class B DIIBF  OTCMKTS  1  OTCQX        0   2017-09-30   

                                                                                                  \
t                                                                                      End(t=36)   
Sharing Economy Firm Incumbent Firms              Ticker Exchange CI Market Index ISF              
Airbnb               Marriott International       MAR    NASDAQ   1  IXIC         0   2008-08-31   
                                                                                  1   2011-08-31   
                     iShares Europe ETF           IEV    NYSEARCA 0  NYA          0   2008-08-31   
                                                                                  1   2011-08-31   
Bird                 Dorel Industries Inc Class B DIIBF  OTCMKTS  1  OTCQX        0   2017-09-30   

                                                                                       

In [9]:
#Drop Multindex
complete_data.columns = ['_'.join(col) for col in complete_data.columns]
complete_data.head()

Date_End(t=1)  \
Sharing Economy Firm Incumbent Firms              Ticker Exchange CI Market Index ISF                 
Airbnb               Marriott International       MAR    NASDAQ   1  IXIC         0      2008-08-31   
                                                                                  1      2008-09-30   
                     iShares Europe ETF           IEV    NYSEARCA 0  NYA          0      2008-08-31   
                                                                                  1      2008-09-30   
Bird                 Dorel Industries Inc Class B DIIBF  OTCMKTS  1  OTCQX        0      2017-09-30   

                                                                                      Date_End(t=12)  \
Sharing Economy Firm Incumbent Firms              Ticker Exchange CI Market Index ISF                  
Airbnb               Marriott International       MAR    NASDAQ   1  IXIC         0       2008-08-31   
                                                                                  1       2009-08-31   
                     iShares Europe ETF           IEV    NYSEARCA 0  NYA          0       2008-08-31   
                                                                                  1       2009-08-31   
Bird                 Dorel Industries Inc Class B DIIBF  OTCMKTS  1  OTCQX        0       2017-09-30   

                                                                                      Date_End(t=24)  \
Sharing Economy Firm Incumbent Firms              Ticker Exchange CI Market Index ISF                  
Airbnb               Marriott International       MAR    NASDAQ   1  IXIC         0       2008-08-31   
                                                                                  1       2010-08-31   
                     iShares Europe ETF           IEV    NYSEARCA 0  NYA          0       2008-08-31   
                                                                                  1       2010-08-31   
Bird                 Dorel Industries Inc Class B DIIBF  OTCMKTS  1  OTCQX        0       2017-09-30   

                                                                                      Date_End(t=3)  \
Sharing Economy Firm Incumbent Firms              Ticker Exchange CI Market Index ISF                 
Airbnb               Marriott International       MAR    NASDAQ   1  IXIC         0      2008-08-31   
                                                                                  1      2008-11-30   
                     iShares Europe ETF           IEV    NYSEARCA 0  NYA          0      2008-08-31   
                                                                                  1      2008-11-30   
Bird                 Dorel Industries Inc Class B DIIBF  OTCMKTS  1  OTCQX        0      2017-09-30   

                                                                                      Date_End(t=36)  \
Sharing Economy Firm Incumbent Firms              Ticker Exchange CI Market Index ISF                  
Airbnb               Marriott International       MAR    NASDAQ   1  IXIC         0       2008-08-31   
                                                                                  1       2011-08-31   
                     iShares Europe ETF           IEV    NYSEARCA 0  NYA          0       2008-08-31   
                                                                                  1       2011-08-31   
Bird                 Dorel Industries Inc Class B DIIBF  OTCMKTS  1  OTCQX        0       2017-09-30   

                                                                                      Date_End(t=6)  \
Sharing Economy Firm Incumbent Firms              Ticker Exchange CI Market Index ISF                 
Airbnb               Marriott International       MAR    NASDAQ   1  IXIC         0      2008-08-31   
                                                                                  1      2009-02-28   
                     iShares Europe ETF    

In [10]:
#Print column names to drop and rename
complete_data.columns

Index(['Date_End(t=1)', 'Date_End(t=12)', 'Date_End(t=24)', 'Date_End(t=3)',
       'Date_End(t=36)', 'Date_End(t=6)', 'Date_Start(t=1)',
       'Date_Start(t=12)', 'Date_Start(t=24)', 'Date_Start(t=3)',
       'Date_Start(t=36)', 'Date_Start(t=6)', 'SP(t)_End(t=1)',
       'SP(t)_End(t=12)', 'SP(t)_End(t=24)', 'SP(t)_End(t=3)',
       'SP(t)_End(t=36)', 'SP(t)_End(t=6)', 'SP(t)_Start(t=1)',
       'SP(t)_Start(t=12)', 'SP(t)_Start(t=24)', 'SP(t)_Start(t=3)',
       'SP(t)_Start(t=36)', 'SP(t)_Start(t=6)', 'Volume(t)_End(t=1)',
       'Volume(t)_End(t=12)', 'Volume(t)_End(t=24)', 'Volume(t)_End(t=3)',
       'Volume(t)_End(t=36)', 'Volume(t)_End(t=6)', 'Volume(t)_Start(t=1)',
       'Volume(t)_Start(t=12)', 'Volume(t)_Start(t=24)',
       'Volume(t)_Start(t=3)', 'Volume(t)_Start(t=36)', 'Volume(t)_Start(t=6)',
       'S&P500(t)_End(t=1)', 'S&P500(t)_End(t=12)', 'S&P500(t)_End(t=24)',
       'S&P500(t)_End(t=3)', 'S&P500(t)_End(t=36)', 'S&P500(t)_End(t=6)',
       'S&P500(t)_Start(t=1)'

In [11]:
#Drop Reset Index/Rename Columns/Drop Start Dates
complete_data = complete_data.reset_index().rename(columns={'SP(t)_End(t=1)':'SP_End(t=1)','SP(t)_End(t=3)':'SP_End(t=3)','SP(t)_End(t=6)':'SP_End(t=6)','SP(t)_End(t=12)':'SP_End(t=12)','SP(t)_End(t=24)':'SP_End(t=24)','SP(t)_End(t=36)':'SP_End(t=36)','SP(t)_Start(t=1)':'SP_Start(t=1)','SP(t)_Start(t=3)':'SP_Start(t=3)','SP(t)_Start(t=6)':'SP_Start(t=6)','SP(t)_Start(t=12)':'SP_Start(t=12)','SP(t)_Start(t=24)':'SP_Start(t=24)','SP(t)_Start(t=36)':'SP_Start(t=36)',
'Volume(t)_End(t=1)':'Volume_End(t=1)','Volume(t)_End(t=3)':'Volume_End(t=3)','Volume(t)_End(t=6)':'Volume_End(t=6)','Volume(t)_End(t=12)':'Volume_End(t=12)','Volume(t)_End(t=24)':'Volume_End(t=24)','Volume(t)_End(t=36)':'Volume_End(t=36)','Volume(t)_Start(t=1)':'Volume_Start(t=1)','Volume(t)_Start(t=3)':'Volume_Start(t=3)','Volume(t)_Start(t=6)':'Volume_Start(t=6)','Volume(t)_Start(t=12)':'Volume_Start(t=12)','Volume(t)_Start(t=24)':'Volume_Start(t=24)','Volume(t)_Start(t=36)':'Volume_Start(t=36)',
'S&P500(t)_End(t=1)':'S&P500_End(t=1)','S&P500(t)_End(t=3)':'S&P500_End(t=3)','S&P500(t)_End(t=6)':'S&P500_End(t=6)','S&P500(t)_End(t=12)':'S&P500_End(t=12)','S&P500(t)_End(t=24)':'S&P500_End(t=24)','S&P500(t)_End(t=36)':'S&P500_End(t=36)','S&P500(t)_Start(t=1)':'S&P500_Start(t=1)','S&P500(t)_Start(t=3)':'S&P500_Start(t=3)','S&P500(t)_Start(t=6)':'S&P500_Start(t=6)','S&P500(t)_Start(t=12)':'S&P500_Start(t=12)','S&P500(t)_Start(t=24)':'S&P500_Start(t=24)','S&P500(t)_Start(t=36)':'S&P500_Start(t=36)'}).drop(columns=['Date_Start(t=1)', 'Date_Start(t=12)', 'Date_Start(t=24)', 'Date_Start(t=3)', 'Date_Start(t=36)', 'Date_Start(t=6)'])
complete_data.head()

,Sharing Economy Firm,Incumbent Firms,Ticker,Exchange,CI,Market Index,ISF,Date_End(t=1),Date_End(t=12),Date_End(t=24),...,S&P500_End(t=24),S&P500_End(t=3),S&P500_End(t=36),S&P500_End(t=6),S&P500_Start(t=1),S&P500_Start(t=12),S&P500_Start(t=24),S&P500_Start(t=3),S&P500_Start(t=36),S&P500_Start(t=6)
0,Airbnb,Marriott International,MAR,NASDAQ,1,IXIC,0,2008-08-31,2008-08-31,2008-08-31,...,1282.83,1282.83,1282.83,1282.83,1267.38,1473.99,1303.82,1400.38,1220.33,1330.63
1,Airbnb,Marriott International,MAR,NASDAQ,1,IXIC,1,2008-09-30,2009-08-31,2010-08-31,...,1049.33,896.24,1218.89,735.09,1282.83,1282.83,1282.83,1282.83,1282.83,1282.83
2,Airbnb,iShares Europe ETF,IEV,NYSEARCA,0,NYA,0,2008-08-31,2008-08-31,2008-08-31,...,1282.83,1282.83,1282.83,1282.83,1267.38,1473.99,1303.82,1400.38,1220.33,1330.63
3,Airbnb,iShares Europe ETF,IEV,NYSEARCA,0,NYA,1,2008-09-30,2009-08-31,2010-08-31,...,1049.33,896.24,1218.89,735.09,1282.83,1282.83,1282.83,1282.83,1282.83,1282.83
4,Bird,Dorel Industries Inc Class B,DIIBF,OTCMKTS,1,OTCQX,0,2017-09-30,2017-09-30,2017-09-30,...,2519.36,2519.36,2519.36,2519.36,2457.59,2168.27,1920.03,2423.41,1972.29,2368.06


In [12]:
#Add Columns for Share Price Growth Rate % for t=(1,3,6,12,24,36)
complete_data['% Change in Share Price (t=1)'] = ((complete_data['SP_End(t=1)']-complete_data['SP_Start(t=1)'])/complete_data['SP_Start(t=1)'])*100
complete_data['% Change in Share Price (t=3)'] = ((complete_data['SP_End(t=3)']-complete_data['SP_Start(t=3)'])/complete_data['SP_Start(t=3)'])*100
complete_data['% Change in Share Price (t=6)'] = ((complete_data['SP_End(t=6)']-complete_data['SP_Start(t=6)'])/complete_data['SP_Start(t=6)'])*100
complete_data['% Change in Share Price (t=12)'] = ((complete_data['SP_End(t=12)']-complete_data['SP_Start(t=12)'])/complete_data['SP_Start(t=12)'])*100
complete_data['% Change in Share Price (t=24)'] = ((complete_data['SP_End(t=24)']-complete_data['SP_Start(t=24)'])/complete_data['SP_Start(t=24)'])*100
complete_data['% Change in Share Price (t=36)'] = ((complete_data['SP_End(t=36)']-complete_data['SP_Start(t=36)'])/complete_data['SP_Start(t=36)'])*100
complete_data.head()

,Sharing Economy Firm,Incumbent Firms,Ticker,Exchange,CI,Market Index,ISF,Date_End(t=1),Date_End(t=12),Date_End(t=24),...,S&P500_Start(t=24),S&P500_Start(t=3),S&P500_Start(t=36),S&P500_Start(t=6),% Change in Share Price (t=1),% Change in Share Price (t=3),% Change in Share Price (t=6),% Change in Share Price (t=12),% Change in Share Price (t=24),% Change in Share Price (t=36)
0,Airbnb,Marriott International,MAR,NASDAQ,1,IXIC,0,2008-08-31,2008-08-31,2008-08-31,...,1303.82,1400.38,1220.33,1330.63,8.876830,-14.281478,-17.272690,-36.492656,-25.093054,-10.742015
1,Airbnb,Marriott International,MAR,NASDAQ,1,IXIC,1,2008-09-30,2009-08-31,2010-08-31,...,1282.83,1282.83,1282.83,1282.83,-7.515141,-40.482228,-49.805155,-14.643252,14.594208,4.927594
2,Airbnb,iShares Europe ETF,IEV,NYSEARCA,0,NYA,0,2008-08-31,2008-08-31,2008-08-31,...,1303.82,1400.38,1220.33,1330.63,-3.969595,-17.391699,-12.361499,-19.760056,-5.200625,14.703657
3,Airbnb,iShares Europe ETF,IEV,NYSEARCA,0,NYA,1,2008-09-30,2009-08-31,2010-08-31,...,1282.83,1282.83,1282.83,1282.83,-12.357080,-35.620053,-47.845207,-19.327177,-25.263852,-20.470536
4,Bird,Dorel Industries Inc Class B,DIIBF,OTCMKTS,1,OTCQX,0,2017-09-30,2017-09-30,2017-09-30,...,1920.03,2423.41,1972.29,2368.06,3.598354,-7.287708,2.181624,-11.176121,1.671961,-20.298333


In [13]:
#Add Columns for Volume Growth Rate % for t=(1,3,6,12,24,36)
complete_data['% Change in Volume (t=1)'] = ((complete_data['Volume_End(t=1)']-complete_data['Volume_Start(t=1)'])/complete_data['Volume_Start(t=1)'])*100
complete_data['% Change in Volume (t=3)'] = ((complete_data['Volume_End(t=3)']-complete_data['Volume_Start(t=3)'])/complete_data['Volume_Start(t=3)'])*100
complete_data['% Change in Volume (t=6)'] = ((complete_data['Volume_End(t=6)']-complete_data['Volume_Start(t=6)'])/complete_data['Volume_Start(t=6)'])*100
complete_data['% Change in Volume (t=12)'] = ((complete_data['Volume_End(t=12)']-complete_data['Volume_Start(t=12)'])/complete_data['Volume_Start(t=12)'])*100
complete_data['% Change in Volume (t=24)'] = ((complete_data['Volume_End(t=24)']-complete_data['Volume_Start(t=24)'])/complete_data['Volume_Start(t=24)'])*100
complete_data['% Change in Volume (t=36)'] = ((complete_data['Volume_End(t=36)']-complete_data['Volume_Start(t=36)'])/complete_data['Volume_Start(t=36)'])*100
complete_data.head()

,Sharing Economy Firm,Incumbent Firms,Ticker,Exchange,CI,Market Index,ISF,Date_End(t=1),Date_End(t=12),Date_End(t=24),...,% Change in Share Price (t=6),% Change in Share Price (t=12),% Change in Share Price (t=24),% Change in Share Price (t=36),% Change in Volume (t=1),% Change in Volume (t=3),% Change in Volume (t=6),% Change in Volume (t=12),% Change in Volume (t=24),% Change in Volume (t=36)
0,Airbnb,Marriott International,MAR,NASDAQ,1,IXIC,0,2008-08-31,2008-08-31,2008-08-31,...,-17.272690,-36.492656,-25.093054,-10.742015,-27.602686,-28.601259,-12.906301,63.017803,75.875492,-52.581639
1,Airbnb,Marriott International,MAR,NASDAQ,1,IXIC,1,2008-09-30,2009-08-31,2010-08-31,...,-49.805155,-14.643252,14.594208,4.927594,219.559358,-45.717575,150.587447,12.873300,42.148715,36.441173
2,Airbnb,iShares Europe ETF,IEV,NYSEARCA,0,NYA,0,2008-08-31,2008-08-31,2008-08-31,...,-12.361499,-19.760056,-5.200625,14.703657,113.633717,-14.839600,-82.271666,-49.248440,38.959852,370.160886
3,Airbnb,iShares Europe ETF,IEV,NYSEARCA,0,NYA,1,2008-09-30,2009-08-31,2010-08-31,...,-47.845207,-19.327177,-25.263852,-20.470536,28.894713,-14.088155,72.496809,-45.839472,727.145834,-67.182963
4,Bird,Dorel Industries Inc Class B,DIIBF,OTCMKTS,1,OTCQX,0,2017-09-30,2017-09-30,2017-09-30,...,2.181624,-11.176121,1.671961,-20.298333,751.861702,10.715520,605.506608,4827.692308,463.908451,47.061524


In [14]:
#Add Columns for S&P500 Growth Rate % for t=(1,3,6,12,24,36)
complete_data['% Change in S&P500 (t=1)'] = ((complete_data['S&P500_End(t=1)']-complete_data['S&P500_Start(t=1)'])/complete_data['S&P500_Start(t=1)'])*100
complete_data['% Change in S&P500 (t=3)'] = ((complete_data['S&P500_End(t=3)']-complete_data['S&P500_Start(t=3)'])/complete_data['S&P500_Start(t=3)'])*100
complete_data['% Change in S&P500 (t=6)'] = ((complete_data['S&P500_End(t=6)']-complete_data['S&P500_Start(t=6)'])/complete_data['S&P500_Start(t=6)'])*100
complete_data['% Change in S&P500 (t=12)'] = ((complete_data['S&P500_End(t=12)']-complete_data['S&P500_Start(t=12)'])/complete_data['S&P500_Start(t=12)'])*100
complete_data['% Change in S&P500 (t=24)'] = ((complete_data['S&P500_End(t=24)']-complete_data['S&P500_Start(t=24)'])/complete_data['S&P500_Start(t=24)'])*100
complete_data['% Change in S&P500 (t=36)'] = ((complete_data['S&P500_End(t=36)']-complete_data['S&P500_Start(t=36)'])/complete_data['S&P500_Start(t=36)'])*100
complete_data.head()

,Sharing Economy Firm,Incumbent Firms,Ticker,Exchange,CI,Market Index,ISF,Date_End(t=1),Date_End(t=12),Date_End(t=24),...,% Change in Volume (t=6),% Change in Volume (t=12),% Change in Volume (t=24),% Change in Volume (t=36),% Change in S&P500 (t=1),% Change in S&P500 (t=3),% Change in S&P500 (t=6),% Change in S&P500 (t=12),% Change in S&P500 (t=24),% Change in S&P500 (t=36)
0,Airbnb,Marriott International,MAR,NASDAQ,1,IXIC,0,2008-08-31,2008-08-31,2008-08-31,...,-12.906301,63.017803,75.875492,-52.581639,1.219050,-8.394150,-3.592283,-12.968880,-1.609885,5.121565
1,Airbnb,Marriott International,MAR,NASDAQ,1,IXIC,1,2008-09-30,2009-08-31,2010-08-31,...,150.587447,12.873300,42.148715,36.441173,-9.079145,-30.135716,-42.697785,-20.439185,-18.201944,-4.984293
2,Airbnb,iShares Europe ETF,IEV,NYSEARCA,0,NYA,0,2008-08-31,2008-08-31,2008-08-31,...,-82.271666,-49.248440,38.959852,370.160886,1.219050,-8.394150,-3.592283,-12.968880,-1.609885,5.121565
3,Airbnb,iShares Europe ETF,IEV,NYSEARCA,0,NYA,1,2008-09-30,2009-08-31,2010-08-31,...,72.496809,-45.839472,727.145834,-67.182963,-9.079145,-30.135716,-42.697785,-20.439185,-18.201944,-4.984293
4,Bird,Dorel Industries Inc Class B,DIIBF,OTCMKTS,1,OTCQX,0,2017-09-30,2017-09-30,2017-09-30,...,605.506608,4827.692308,463.908451,47.061524,2.513438,3.959297,6.389196,16.192172,31.214616,27.737807


In [15]:
#Convert Exchange to Dummy Variables for either NYSE, NASDAQ, or Other (OTCMKTS, TWSE, and NSE)
exchange_dummies = pd.get_dummies(complete_data['Exchange'])
complete_data = pd.concat([complete_data,exchange_dummies],axis=1)
complete_data['Other'] = complete_data['NYSEARCA']+complete_data['OTCMKTS']
complete_data['Sharing Firm Introduced'] = np.where(complete_data['ISF'] == 1, "Yes", "No")
complete_data['Relatedness with Sharing Firm'] = np.where(complete_data['CI'] == 1, "Closely Related", "Less Closely Related")
complete_data = complete_data.drop(columns=['NYSEARCA','OTCMKTS']).sort_values(by=['Sharing Economy Firm','CI'],ascending=[True, False])
complete_data.head()

,Sharing Economy Firm,Incumbent Firms,Ticker,Exchange,CI,Market Index,ISF,Date_End(t=1),Date_End(t=12),Date_End(t=24),...,% Change in S&P500 (t=3),% Change in S&P500 (t=6),% Change in S&P500 (t=12),% Change in S&P500 (t=24),% Change in S&P500 (t=36),NASDAQ,NYSE,Other,Sharing Firm Introduced,Relatedness with Sharing Firm
0,Airbnb,Marriott International,MAR,NASDAQ,1,IXIC,0,2008-08-31,2008-08-31,2008-08-31,...,-8.394150,-3.592283,-12.968880,-1.609885,5.121565,1,0,0,No,Closely Related
1,Airbnb,Marriott International,MAR,NASDAQ,1,IXIC,1,2008-09-30,2009-08-31,2010-08-31,...,-30.135716,-42.697785,-20.439185,-18.201944,-4.984293,1,0,0,Yes,Closely Related
2,Airbnb,iShares Europe ETF,IEV,NYSEARCA,0,NYA,0,2008-08-31,2008-08-31,2008-08-31,...,-8.394150,-3.592283,-12.968880,-1.609885,5.121565,0,0,1,No,Less Closely Related
3,Airbnb,iShares Europe ETF,IEV,NYSEARCA,0,NYA,1,2008-09-30,2009-08-31,2010-08-31,...,-30.135716,-42.697785,-20.439185,-18.201944,-4.984293,0,0,1,Yes,Less Closely Related
4,Bird,Dorel Industries Inc Class B,DIIBF,OTCMKTS,1,OTCQX,0,2017-09-30,2017-09-30,2017-09-30,...,3.959297,6.389196,16.192172,31.214616,27.737807,0,0,1,No,Closely Related


In [16]:
#Write complete data to Excel and CSV
with pd.ExcelWriter('Data/complete_data.xlsx',engine='openpyxl') as writer:
    complete_data.to_excel(writer,index=False)
complete_data.to_csv('Data/complete_data.csv',index=False)

In [17]:
#Create STATA ready data
stata_data = complete_data[['CI', 'ISF', 'NASDAQ', 'NYSE', 'Other', '% Change in Volume (t=1)', '% Change in Volume (t=3)', '% Change in Volume (t=6)', '% Change in Volume (t=12)',  '% Change in Volume (t=24)',  '% Change in Volume (t=36)', '% Change in Share Price (t=1)', '% Change in Share Price (t=3)', '% Change in Share Price (t=6)', '% Change in Share Price (t=12)', '% Change in Share Price (t=24)', '% Change in Share Price (t=36)', '% Change in S&P500 (t=1)', '% Change in S&P500 (t=3)', '% Change in S&P500 (t=6)', '% Change in S&P500 (t=12)', '% Change in S&P500 (t=24)', '% Change in S&P500 (t=36)','Date_End(t=1)', 'Date_End(t=12)', 'Date_End(t=24)', 'Date_End(t=3)', 'Date_End(t=36)', 'Date_End(t=6)', 'Sharing Firm Introduced','Relatedness with Sharing Firm']].rename(columns={'% Change in Volume (t=1)':'one_m_diff_volume', '% Change in Volume (t=3)':'three_m_diff_volume', '% Change in Volume (t=6)':'six_m_diff_volume', '% Change in Volume (t=12)':'one_y_diff_volume', '% Change in Volume (t=24)':'two_y_diff_volume', '% Change in Volume (t=36)':'three_y_diff_volume', '% Change in Share Price (t=1)':'one_m_diff_shareprice', '% Change in Share Price (t=3)':'three_m_diff_shareprice', '% Change in Share Price (t=6)':'six_m_diff_shareprice', '% Change in Share Price (t=12)':'one_y_diff_shareprice', '% Change in Share Price (t=24)':'two_y_diff_shareprice', '% Change in Share Price (t=36)':'three_y_diff_shareprice', '% Change in S&P500 (t=1)':'one_m_diff_sp500', '% Change in S&P500 (t=3)':'three_m_diff_sp500', '% Change in S&P500 (t=6)':'six_m_diff_sp500', '% Change in S&P500 (t=12)':'one_y_diff_sp500', '% Change in S&P500 (t=24)':'two_y_diff_sp500', '% Change in S&P500 (t=36)':'three_y_diff_sp500'})
stata_data['Exchange'] = stata_data[['NASDAQ','NYSE','Other']].idxmax(axis=1)
stata_data.head()

,CI,ISF,NASDAQ,NYSE,Other,one_m_diff_volume,three_m_diff_volume,six_m_diff_volume,one_y_diff_volume,two_y_diff_volume,...,three_y_diff_sp500,Date_End(t=1),Date_End(t=12),Date_End(t=24),Date_End(t=3),Date_End(t=36),Date_End(t=6),Sharing Firm Introduced,Relatedness with Sharing Firm,Exchange
0,1,0,1,0,0,-27.602686,-28.601259,-12.906301,63.017803,75.875492,...,5.121565,2008-08-31,2008-08-31,2008-08-31,2008-08-31,2008-08-31,2008-08-31,No,Closely Related,NASDAQ
1,1,1,1,0,0,219.559358,-45.717575,150.587447,12.873300,42.148715,...,-4.984293,2008-09-30,2009-08-31,2010-08-31,2008-11-30,2011-08-31,2009-02-28,Yes,Closely Related,NASDAQ
2,0,0,0,0,1,113.633717,-14.839600,-82.271666,-49.248440,38.959852,...,5.121565,2008-08-31,2008-08-31,2008-08-31,2008-08-31,2008-08-31,2008-08-31,No,Less Closely Related,Other
3,0,1,0,0,1,28.894713,-14.088155,72.496809,-45.839472,727.145834,...,-4.984293,2008-09-30,2009-08-31,2010-08-31,2008-11-30,2011-08-31,2009-02-28,Yes,Less Closely Related,Other
4,1,0,0,0,1,751.861702,10.715520,605.506608,4827.692308,463.908451,...,27.737807,2017-09-30,2017-09-30,2017-09-30,2017-09-30,2017-09-30,2017-09-30,No,Closely Related,Other


In [18]:
#Write STATA data to Excel and CSV
with pd.ExcelWriter('Data/stata_data.xlsx',engine='openpyxl') as writer:
    stata_data.to_excel(writer,index=False,)
stata_data.to_csv('Data/stata_data.csv',index=False)